# 情感分析：使用循环神经网络
:label:`sec_sentiment_rnn`

下面这段话表达的意思是：
借鉴迁移学习的思想：利用外部大规模语料GloVe中学到的通用语言知识，提升小数据集IMDb上的模型泛化能力。GloVe 是一种通过全局词共现统计来学习高质量词向量的方法，学习高质量的词向量，使语义相近的词在向量空间中距离更近，广泛用于各种自然语言处理任务中作为语义表示的基础。

GloVe 和 Word2Vec 是两种不同的词向量学习方法：Word2Vec 基于局部上下文预测，GloVe 基于全局共现统计；它们殊途同归，都是现代 NLP 中词表示的重要基石。

Word2Vec、GloVe 是“静态”的：每个词只有一个固定向量，不管它出现在什么句子中。例如：“bank” 在 “river bank” 和 “bank account” 中含义不同，GloVe 给的都是同一个向量。

BERT 是“动态”的：同一个词在不同句子中会有不同的向量表示。它能区分 “bank” 是“河岸”还是“银行”。

在 IMDb 情感分析任务中：
你可以用预训练的 GloVe 向量来表示评论中的每个词；然后把这些向量输入 RNN、CNN 或 Transformer 等模型；这样模型就不用从零学习“好”“棒”“差”“烂”这些词的含义，而是直接利用 GloVe 已经学到的语义信息。

与词相似度和类比任务一样，我们也可以将预先训练的词向量应用于情感分析。由于 :numref:`sec_sentiment`中的IMDb评论数据集不是很大，使用在大规模语料库上预训练的文本表示可以减少模型的过拟合。作为 :numref:`fig_nlp-map-sa-rnn`中所示的具体示例，我们将使用预训练的GloVe模型来表示每个词元，并将这些词元表示送入多层双向循环神经网络以获得文本序列表示，该文本序列表示将被转换为情感分析输出 :cite:`Maas.Daly.Pham.ea.2011`。对于相同的下游应用，我们稍后将考虑不同的架构选择。

![将GloVe送入基于循环神经网络的架构，用于情感分析](../img/nlp-map-sa-rnn.svg)
:label:`fig_nlp-map-sa-rnn`


In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 64
train_iter, test_iter, vocab = d2l.load_data_imdb(batch_size)

In [ ]:
for batch in train_iter:
    X, y = batch
    print(X, y)
    break


## 使用循环神经网络表示单个文本

在文本分类任务（如情感分析）中，可变长度的文本序列将被转换为固定长度的类别。在下面的`BiRNN`类中，虽然文本序列的每个词元经由嵌入层（`self.embedding`）获得其单独的预训练GloVe表示，但是整个序列由双向循环神经网络（`self.encoder`）编码。更具体地说，双向长短期记忆网络在初始和最终时间步的隐状态（在最后一层）被连结起来作为文本序列的表示。然后，通过一个具有两个输出（“积极”和“消极”）的全连接层（`self.decoder`），将此单一文本表示转换为输出类别。


In [ ]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs)

        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 将bidirectional设置为True以获取双向循环神经网络
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, bidirectional=True)
        self.decoder = nn.Linear(4 * num_hiddens, 2)

    # inputs(B,Q)
    def forward(self, inputs):
        # inputs的形状是（批量大小，时间步数）
        # 因为长短期记忆网络要求其输入的第一个维度是时间维，
        # 所以在获得词元表示之前，输入会被转置。
        # 输出形状为（时间步数，批量大小，词向量维度）
        # inputs(B,Q)->(Q,B)
        # embeddings(Q,B,E)
        embeddings = self.embedding(inputs.T)
        # 将LSTM参数展平以提高GPU计算效率
        self.encoder.flatten_parameters()
        # outputs(Q,B,2H):所有时间步上 最后一层 的隐状态
        outputs, _ = self.encoder(embeddings)
        # 连结初始和最终时间步的隐状态，作为全连接层的输入，
        # 其形状为（批量大小，4*隐藏单元数）
        # outputs[0](B,2H)
        # outputs[-1](B,2H)
        # encoding(B,4H))
        encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
        # outs(B,2)
        outs = self.decoder(encoding)
        return outs

让我们构造一个具有两个隐藏层的双向循环神经网络来表示单个文本以进行情感分析。


In [ ]:
embed_size, num_hiddens, num_layers = 100, 100, 2
devices = d2l.try_all_gpus()
net = BiRNN(len(vocab), embed_size, num_hiddens, num_layers)

In [ ]:
# Xavier 初始化（Glorot 初始化）适用于使用 tanh 或 sigmoid 激活函数 的网络，能保持前向传播时激活值和反向传播时梯度的方差稳定。
def init_weights(m):
    # 嵌入曾被冻结了，这里不做初始化
    # 线形层
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    # RNN(LSTM) 层
    if type(m) == nn.LSTM:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])
net.apply(init_weights);

In [ ]:
# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight)
#         if m.bias is not None:
#             nn.init.zeros_(m.bias)
#     elif isinstance(m, nn.LSTM):
#         for name, param in m.named_parameters():
#             if "weight" in name:
#                 nn.init.xavier_uniform_(param)
#             elif "bias" in name:
#                 nn.init.zeros_(param)
#     # 可选：初始化 Embedding
#     # elif isinstance(m, nn.Embedding):
#     #     nn.init.uniform_(m.weight, -0.1, 0.1)

## 加载预训练的词向量

下面，我们为词表中的单词加载预训练的100维（需要与`embed_size`一致）的GloVe嵌入。


In [ ]:
# 预训练好的词元到向量的映射
glove_embedding = d2l.TokenEmbedding('glove.6b.100d')

打印词表中所有词元向量的形状。


In [ ]:
print(vocab.idx_to_token[0:10])
print(glove_embedding[['the']])
print(glove_embedding[['The']])


In [ ]:
# 从预训练好的映射表中提取与词汇表中所有词元对应的向量，
# 并将这些向量堆叠成一个矩阵 embeds，
# 其中 embeds[i] 是词汇表中第 i 个词元的预训练向量。
# vocab.idx_to_token(V)
# embeds(V,E)
embeds = glove_embedding[vocab.idx_to_token]
print(embeds.shape)

我们使用这些预训练的词向量来表示评论中的词元，并且在训练期间不要更新这些向量。


In [ ]:
# 将预训练好的词向量 embeds 加载到模型 net 的嵌入层作为初始权重。
net.embedding.weight.data.copy_(embeds)
# 冻结 嵌入层的权重，使其在模型训练过程中保持不变。
net.embedding.weight.requires_grad = False

## 训练和评估模型

现在我们可以训练双向循环神经网络进行情感分析。


In [ ]:
lr, num_epochs = 0.01, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction="none")
d2l.train_ch13(net, train_iter, test_iter, loss, optimizer, num_epochs, devices)


我们定义以下函数来使用训练好的模型`net`预测文本序列的情感。


In [ ]:
#@save
def predict_sentiment(net, vocab, sequence):
    """预测文本序列的情感"""
    sequence = torch.tensor(vocab[sequence.split()], device=d2l.try_gpu())
    label = torch.argmax(net(sequence.reshape(1, -1)), dim=1)
    return 'positive' if label == 1 else 'negative'

最后，让我们使用训练好的模型对两个简单的句子进行情感预测。


In [ ]:
predict_sentiment(net, vocab, 'this movie is so great')

In [ ]:
predict_sentiment(net, vocab, 'this movie is so bad')

## 小结

* 预训练的词向量可以表示文本序列中的各个词元。
* 双向循环神经网络可以表示文本序列。例如通过连结初始和最终时间步的隐状态，可以使用全连接的层将该单个文本表示转换为类别。

## 练习

1. 增加迭代轮数可以提高训练和测试的准确性吗？调优其他超参数怎么样？
1. 使用较大的预训练词向量，例如300维的GloVe嵌入。它是否提高了分类精度？
1. 是否可以通过spaCy词元化来提高分类精度？需要安装Spacy（`pip install spacy`）和英语语言包（`python -m spacy download en`）。在代码中，首先导入Spacy（`import spacy`）。然后，加载Spacy英语软件包（`spacy_en = spacy.load('en')`）。最后，定义函数`def tokenizer(text): return [tok.text for tok in spacy_en.tokenizer(text)]`并替换原来的`tokenizer`函数。请注意GloVe和spaCy中短语标记的不同形式。例如，短语标记“new york”在GloVe中的形式是“new-york”，而在spaCy词元化之后的形式是“new york”。


[Discussions](https://discuss.d2l.ai/t/5724)
